In [1]:
from pathlib import Path
import re
from collections import Counter

In [ ]:
# mattermost 메세지를 시간 기준으로 나누는 함수

# "오전 1:07", "오후 12:30" 같은 시간 패턴
TIME_RE = re.compile(r'^(오전|오후)\s\d{1,2}:\d{2}$')

def split_mm_messages(raw: str):
    lines = [l.rstrip() for l in raw.splitlines()]
    messages = []
    cur = []

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # 현재 줄 다음 줄이 '오전/오후 HH:MM' 이면 -> 새 메시지 헤더
        if i + 1 < len(lines) and TIME_RE.match(lines[i+1].strip()):
            # 지금까지 모인 내용이 있으면 하나의 메시지로 저장
            if cur:
                messages.append("\n".join(cur).strip())
                cur = []
            # 이름 줄 + 시간 줄 두 줄은 건너뛰기
            i += 2
            continue

        # '시스템' 같은 라벨이나 완전 빈 줄은 건너뛰기
        if line == "" or line == "시스템":
            i += 1
            continue

        # 나머지는 현재 메시지 내용으로 추가
        cur.append(lines[i])
        i += 1

    # 마지막 메시지 처리
    if cur:
        messages.append("\n".join(cur).strip())

    return messages

In [ ]:
# mattermost 메세지들 전처리 함수

EMOJI_RE = re.compile(r':[0-9A-Za-z_+\-]+:')          # Mattermost 스타일 이모지
URL_RE   = re.compile(r'http[s]?://\S+')              # url
MENTION_RE = re.compile(r'@\S+')                      # mention

def clean_message(msg: str) -> str:
    lines = []
    for line in msg.splitlines():
        line = line.strip()
        if not line:
            continue

        # 1) URL → <URL>
        line = URL_RE.sub(' <URL> ', line)

        # 2) 맨션(@all, @이름) → <MENTION>
        line = MENTION_RE.sub(' <MENTION> ', line)

        # 3) 이모지 :something: 제거
        line = EMOJI_RE.sub(' ', line)

        # 4) 괄호 제거
        for ch in ['[', ']', '(', ')', '{', '}', '“', '”', '"', "'", "<", ">"]:
            line = line.replace(ch, ' ')

        # 5) 이모티콘 문자들은 전부 공백 처리 (▶▶, ★, … 등)
        line = re.sub(r'[^0-9A-Za-z가-힣<>\s]', ' ', line)

        # 6) 여러 공백 → 하나
        line = re.sub(r'\s+', ' ', line).strip()

        if line:
            lines.append(line)

    # 한 메시지를 한 줄로 합치기
    return " ".join(lines)

In [ ]:
# 파일 읽어오기
path = Path("../corpus/mattermost/mm.txt")
raw_text = path.read_text(encoding="utf-8")

# mm 메세지 나누기
messages = split_mm_messages(raw_text)
len(messages), messages[0], messages[3]

# mm 메세지 전처리
cleaned_messages = [clean_message(m) for m in messages]
len(cleaned_messages), cleaned_messages[0]

In [ ]:
# 새로운 파일로 작성
out_path = Path("../corpus/mm_total.txt")

with out_path.open("w", encoding="utf-8") as f:
    for msg in cleaned_messages:
        msg = msg.strip()
        if not msg:
            continue
        f.write(msg + "\n")

out_path

## 잘 뽑혔는지 확인해보자.

In [2]:
# 1) 코퍼스 파일 다시 읽기
path = Path("../corpus/mm_total.txt")
lines = path.read_text(encoding="utf-8").splitlines()

# 공백 줄 제거 + 좌우 공백 제거
lines = [ln.strip() for ln in lines if ln.strip()]

print("문장(메시지) 수:", len(lines))

# 2) 토큰 나누기 (띄어쓰기 기준)
tokens = []
for ln in lines:
    tokens.extend(ln.split())

print("전체 토큰 개수:", len(tokens))
print("서로 다른 단어 개수:", len(set(tokens)))

# 3) 가장 많이 나온 단어 50개만 찍어보기
counter = Counter(tokens)
for w, c in counter.most_common(50):
    print(w, c)

문장(메시지) 수: 597
전체 토큰 개수: 37545
서로 다른 단어 개수: 6279
싸피 581
안내 396
및 281
수 281
안녕하세요 237
바랍니다 227
14기 207
00 195
신청 168
채용설명회 167
참여 158
평가 156
11 149
10 145
통해 144
제출 144
설문 140
후 137
11월 137
12 136
감사합니다 131
에듀싸피 131
8월 129
9월 127
멘토링 116
필수 114
취업특강 113
공지사항 110
월 107
10월 107
있습니다 105
이벤트 103
반 102
여러분 101
온라인 99
반드시 97
클릭 96
시 96
인공지능 96
있는 95
중 94
30 94
8 94
함께 93
응시 92
IT 91
1 89
목 89
화 88
채용 88
